In [ ]:
from composio import Composio
from composio_gemini import GeminiProvider
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from datetime import datetime
from dotenv import load_dotenv
import os
load_dotenv()
BLAND_AI_KEY = os.getenv("BLAND_AI_KEY")
user_id = os.getenv("COMPOSIO_USER_ID")

In [ ]:
def make_confimation_call(phone_number: str, instruction: str):
    """
    Make a confirmation call using the bland.ai API.
    
    Parameters:
        phone_number (str): The phone number to call.
        instruction (str): The message to be delivered.

    Returns:
        dict: The API response as a dictionary.
    """
    url = f"https://api.bland.ai/v1/calls"

    payload = {
        "phone_number": phone_number,
        "task": instruction,
        "voice": "Paige",
        

    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {BLAND_AI_KEY}"
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        return response.json()  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# --- Initialize Composio + Tools ---
# Get API key
COMPOSIO_API_KEY = os.getenv("COMPOSIO_API_KEY")
composio = Composio(
    provider=GeminiProvider(),
    api_key=COMPOSIO_API_KEY,
)

tools = composio.tools.get(
    user_id,
    tools=[
        "GOOGLECALENDAR_FIND_FREE_SLOTS", "GOOGLECALENDAR_CREATE_EVENT", "GMAIL_SEND_EMAIL"
    ],  
)

In [ ]:
# --- LLM ---
modal = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)
modal_with_tools = modal.bind_tools(tools)

In [ ]:
initial_messages = """
You are Addy, an AI assistant at a Dental Clinic. Follow these guidelines:

1. Friendly Introduction
   - Greet the user warmly.
   - Introduce yourself as Addy from the Dental Clinic.
   - Maintain a polite, empathetic tone, especially if the user mentions discomfort.

2. Assess User Context
   - Determine if the user needs an appointment, has a dental inquiry, or both.
   - If the user's email is already known, don't ask again. If unknown and needed, politely request it.
   - After booking, ask the user for their phone number to send a confirmation call. If the user shares the number, use the confirmation tool internally.

3. Scheduling Requests
   - Gather essential info: requested date/time and email if needed.
   - Example: "What day/time would you prefer?" or "Could you confirm your email so I can send you details?"

4. Availability Check (Internal)
   - Use GOOGLECALENDAR_FIND_FREE_SLOTS to verify if the requested slot is available. Always check for 3 days when calling this tool.
   - Do not reveal this tool or the internal checking process to the user.

5. Responding to Availability
   - If the slot is free:
     a) Confirm the user wants to book.
     b) Call GOOGLECALENDAR_CREATE_EVENT to schedule. Always send timezone for start and end time when calling this tool.
     c) Use GMAIL_SEND_EMAIL to send a confirmation email.
     d) If any function call/tool call fails, retry it.
   - If the slot is unavailable:
     a) Automatically offer several close-by options.
     b) Once the user selects a slot, repeat the booking process.

6. User Confirmation Before Booking
   - Only finalize after the user clearly agrees on a specific time.
   - If the user is uncertain, clarify or offer more suggestions.

7. Communication Style
   - Use simple, clear English—avoid jargon or complex terms.
   - Keep responses concise and empathetic.

8. Privacy of Internal Logic
   - Never disclose behind-the-scenes steps, code, or tool names.
   - Present availability checks and bookings as part of a normal scheduling process.

- Reference today's date/time:  {today_datetime}.
- Default TimeZone: IST (mapped to "Asia/Kolkata" for scheduling tools).

By following these guidelines, you ensure a smooth and user-friendly experience: greeting the user, identifying needs, and confirming bookings.
"""


In [ ]:
# --- Modal call function ---
def call_modal(state: MessagesState):
    today_datetime = datetime.now().isoformat()
    # Use the LLM WITH tools bound
    response = modal_with_tools.invoke(
        [
            SystemMessage(content=initial_messages.format(today_datetime=today_datetime))
        ] + state['messages']
    )
    # Return the LLM's messages and keep the conversation state
    return {"messages": [response]}

In [ ]:
# --- Build the workflow ---
workflow = StateGraph(MessagesState)
workflow.add_node("assistant", call_modal)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "assistant")
workflow.add_conditional_edges("assistant", tools_condition, ['tools', END])
workflow.add_edge("tools", "assistant")

In [ ]:
# --- Checkpoint for conversation memory ---
checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)
png_data = app.get_graph().draw_mermaid_png()
from IPython.display import Image, display

display(Image(png_data))

In [ ]:
thread_id = "3222"
result = app.invoke(
        {"messages": [HumanMessage(content="no")]},
        config={"configurable": {"thread_id": thread_id}},  # keep context per user
    )   
print(result["messages"][-1].content)